In [1]:
import pysam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [2]:
import splanl.junction_scorer as jn
import splanl.coords as cds
import splanl.merge_bcs as mbcs
import splanl.plots as sp
import splanl.score_motifs as sm
import splanl.inspect_variants as iv
import splanl.post_processing as pp
import splanl.gnomad as gn

Using TensorFlow backend.


#this didn't work in the notebook
#probably a version issue with tabix?
! module load samtools_and_tabix/1.3

! cd /nfs/kitzman2/smithcat/refs/gnomAD/

! tabix -h -p  vcf   http://storage.googleapis.com/gcp-public-data--gnomad/release/2.1.1/vcf/genomes/gnomad.genomes.r2.1.1.sites.vcf.bgz   11:32413470-32413660 > /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v2.vcf

#hg38 coords for exon are 21,546 bases lower than hg19 coords
! tabix -h -p  vcf   http://storage.googleapis.com/gcp-public-data--gnomad/release/3.1.2/vcf/genomes/gnomad.genomes.v3.1.2.sites.chr11.vcf.bgz  chr11:32391924-32392114  > /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v3.vcf

In [3]:
#zip file
! bgzip -c /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v2.vcf > /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v2.vcf.gz

In [4]:
#index file
! tabix -p vcf /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v2.vcf.gz

In [5]:
#zip file
! bgzip -c /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v3.vcf > /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v3.vcf.gz

In [6]:
#index file
! tabix -p vcf /nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v3.vcf.gz

In [7]:
gnomad2_tbx = pysam.TabixFile( '/nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v2.vcf.gz' )

In [8]:
gnomad3_tbx = pysam.TabixFile( '/nfs/kitzman2/smithcat/refs/gnomAD/gnomad_wt1_v3.vcf.gz' )

In [9]:
gnomad2_df = gn.create_gnomad_df( gnomad2_tbx,
                                 '11',
                                 ( 32413470, 32413660 ), ).rename( columns = { 'gdna_pos_hg38': 'hg19_pos', } )

In [10]:
gnomad3_df = gn.create_gnomad_df( gnomad3_tbx,
                                 'chr11',
                                 ( 32391924, 32392114 ), )

In [11]:
gnomad2_df

,chrom,hg19_pos,ref,alt,n_alt,n_allele,n_homo,af
0,11,32413476,A,T,2,31402,0,0.000064
1,11,32413531,A,G,1,31402,0,0.000032
2,11,32413545,G,A,1,27976,0,0.000036
3,11,32413547,G,A,2,27640,0,0.000072
4,11,32413549,CT,C,1,31350,0,0.000032
5,11,32413552,CA,C,1,27992,0,0.000036
6,11,32413555,G,A,1,31382,0,0.000032
7,11,32413561,G,A,1,29974,0,0.000033
8,11,32413561,GGA,G,0,29974,0,0.000000
9,11,32413566,GGGA,G,1,29780,0,0.000034


In [12]:
gnomad3_df[ 'hg19_pos' ] = gnomad3_df.gdna_pos_hg38 + 21546

In [13]:
gnomad3_df

,chrom,gdna_pos_hg38,ref,alt,n_alt,n_allele,n_homo,af,hg19_pos
0,chr11,32391927,T,C,1,152230,0,0.000007,32413473
1,chr11,32391929,TA,T,1,152178,0,0.000007,32413475
2,chr11,32391930,A,T,7,152230,0,0.000046,32413476
3,chr11,32391942,A,C,1,152248,0,0.000007,32413488
4,chr11,32391952,G,A,1,152176,0,0.000007,32413498
5,chr11,32391985,A,G,2,152216,0,0.000013,32413531
6,chr11,32391995,G,A,1,152114,0,0.000007,32413541
7,chr11,32391999,G,A,59,122380,0,0.000482,32413545
8,chr11,32392001,G,A,58,121590,0,0.000477,32413547
9,chr11,32392003,CT,C,57,151526,0,0.000376,32413549


In [14]:
bdout = '/nfs/kitzman2/smithcat/proj/wt1_2022/ex9_data/'

In [15]:
gnomad2_df.to_csv( bdout + 'gnomad_v2_wt1.txt',
                   sep = '\t',
                   index=False
                )

In [16]:
gnomad3_df.to_csv( bdout + 'gnomad_v3_wt1.txt',
                   sep = '\t',
                   index=False
                )